# 모델 앙상블

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input, Reshape

import time

# 회귀 문제

In [2]:
x = np.arange(-1, 1, 0.01)
np.random.shuffle(x)
y = np.sin(x)

i = int(x.shape[0]*0.8)

train_x, test_x = x[:i], x[i:]
train_y, test_y = y[:i], y[i:]


In [19]:
def build_model_A():  
  model = keras.Sequential()
  model.add(Input((1,)))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1))

  model.compile(optimizer="Adam", loss="mse")

  return model
  
def build_model_B():  
  model = keras.Sequential()
  model.add(Input((1,)))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1))

  model.compile(optimizer="Adam", loss="mse")

  return model

def build_model_C():  
  model = keras.Sequential()
  model.add(Input((1,)))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1))

  model.compile(optimizer="Adam", loss="mse")

  return model

In [20]:
models = []
models.append(build_model_A())
models.append(build_model_B())
models.append(build_model_C())

In [21]:
for i in range(len(models)):
	models[i].fit(train_x, train_y, epochs=1, batch_size=32)

5/5 [==============================] - 0s 3ms/step - loss: 0.2589


In [22]:
y_ = [model.predict(test_x) for model in models]
y_ = np.array(y_)
print("y_.shape =", y_.shape)
outcomes = np.mean(y_,axis=0).squeeze()
print("outcomes.shape =", outcomes.shape)

mse = tf.keras.losses.mean_squared_error(test_y, outcomes).numpy()
print("mse =", mse)

y_.shape = (3, 40, 1)
outcomes.shape = (40,)
mse = 0.33232737


# 분류 문제

In [23]:
(raw_train_x, raw_train_y), (raw_test_x, raw_test_y) = tf.keras.datasets.mnist.load_data()

train_x = raw_train_x / 255.
test_x = raw_test_x / 255.

train_y = raw_train_y
test_y = raw_test_y

11501568/11490434 [==============================] - 0s 0us/step


In [24]:
def build_model():  
  model = keras.Sequential()
  model.add(Input((28,28)))
  model.add(Reshape((28,28,1)))
  model.add(Conv2D(32, (3, 3), padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

  return model

In [25]:
n_members = 3

In [26]:
models = []
for _ in range(n_members):
	model = build_model()
	models.append(model)

In [27]:
for i in range(n_members):
	models[i].fit(train_x, train_y, epochs=1, batch_size=32)

1875/1875 [==============================] - 6s 3ms/step - loss: 0.3468 - accuracy: 0.8963


In [29]:
from sklearn.metrics import accuracy_score

y_ = [model.predict(test_x) for model in models]
y_ = np.array(y_)
summed = np.sum(y_, axis=0)
print(summed[0])
outcomes = np.argmax(summed, axis=1)
print(outcomes)

acc = accuracy_score(test_y, outcomes)
print(acc)


[3.2679769e-03 2.3581135e-05 1.3942655e-02 5.2133917e-05 2.5485101e-04
 9.4831563e-07 2.7479923e-09 2.9808586e+00 7.3838508e-04 8.6125883e-04]
[7 2 1 ... 4 5 6]
0.9756


# 2진 분류 문제

In [30]:
!wget https://raw.githubusercontent.com/dhrim/MDC_2021/master/material/deep_learning/data/sonar.csv

--2021-12-29 00:39:44--  https://raw.githubusercontent.com/dhrim/MDC_2021/master/material/deep_learning/data/sonar.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87776 (86K) [text/plain]
Saving to: ‘sonar.csv’

sonar.csv           100%[===================>]  85.72K  --.-KB/s    in 0.01s   

2021-12-29 00:39:44 (6.25 MB/s) - ‘sonar.csv’ saved [87776/87776]



In [31]:
raw_data = pd.read_csv("sonar.csv")
data = raw_data.to_numpy()
np.random.shuffle(data)

x = data[:,:60]
y = data[:,60]

i = int(len(x)*0.8)
train_x, test_x = x[:i], x[i:]
train_y, test_y = y[:i], y[i:]

In [32]:
def build_model():  

  model = keras.Sequential()
  model.add(Dense(10, activation='relu', input_shape=(60,)))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation="sigmoid"))

  model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["acc"])

  return model

In [33]:
n_members = 3

In [34]:
models = []
for _ in range(n_members):
	model = build_model()
	models.append(model)

In [35]:
for i in range(n_members):
	models[i].fit(train_x, train_y, epochs=1, batch_size=32, validation_split=0.2)

5/5 [==============================] - 1s 39ms/step - loss: 0.7163 - acc: 0.5379 - val_loss: 0.7580 - val_acc: 0.3333


In [51]:
y_ = [model.predict(test_x[:10]) for model in models]
y_ = np.array(y_).squeeze()
predicted = (y_>0.5).astype(np.int)
print(predicted)
print("->")
outcomes, _ = scipy.stats.mode(predicted)
print(outcomes[:10])

[[0 0 0 1 0 1 0 0 1 0]
 [1 0 1 1 1 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0]]
->
[[0 0 0 1 0 0 0 0 0 0]]


In [44]:
y_ = [model.predict(test_x) for model in models]
y_ = np.array(y_).squeeze()
predicted = (y_>0.5).astype(np.int)

import scipy
outcomes, _ = scipy.stats.mode(predicted)
print(outcomes[:10])
outcomes = outcomes.squeeze()


from sklearn.metrics import accuracy_score

acc = accuracy_score(test_y, outcomes)
print("acc = ", acc)

[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
  0 0 0 0 1 0]]
acc =  0.42857142857142855
